In [4]:
import numpy as np
from tensorflow import keras
import tensorflow as tf


class MNN(keras.Model):
    def __init__(self, node_size, nhid0, nhid1, droput, alpha):
        super(MNN, self).__init__()
        self.encode0 = tf.keras.layers.Dense(node_size, nhid0)
        self.encode1 = tf.keras.layers.Dense(nhid0, nhid1)
        self.decode0 = tf.keras.layers.Dense(nhid1, nhid0)
        self.decode1 = tf.keras.layers.Dense(nhid0, node_size)
        self.droput = droput
        self.alpha = alpha

    def call(self, adj_batch, adj_mat, b_mat):
        t0 = tf.keras.layers.LeakyReLU(self.encode0(adj_batch))
        t0 = tf.keras.layers.LeakyReLU(self.encode1(t0))
        embedding = t0
        t0 = tf.keras.layers.LeakyReLU(self.decode0(t0))
        t0 = tf.keras.layers.LeakyReLU(self.decode1(t0))
        embedding_norm = tf.reduce_sum(embedding * embedding, axis=1, keepdims=True)
        L_1st = tf.reduce_sum(adj_mat * (embedding_norm -
                                     2 * tf.matmul(embedding, tf.transpose(embedding))
                                     + tf.transpose(embedding_norm)))
        L_2nd = tf.reduce_sum(((adj_batch - t0) * b_mat) * ((adj_batch - t0) * b_mat))
        return L_1st, self.alpha * L_2nd, L_1st + self.alpha * L_2nd

    def savector(self, adj):
        t0 = self.encode0(adj)
        t0 = self.encode1(t0)
        return t0


In [8]:
import networkx as nx
import numpy as np

In [16]:

def Read_graph(file_name):
    # 文本文件中的每一行必须含有相同的数据; delimiter分隔符默认是空格; 类型是numpy array
    edge = np.loadtxt(file_name).astype(np.int32)
    # 得到图中点的最小和最大编号; .min()返回数组中所有元素最小的
    min_node, max_node = edge.min(), edge.max()
    # Node表示图上一共有多少个顶点，如果标号是从0开始，那么顶点数 = max_node + 1
    if min_node == 0:
        Node = max_node + 1
    else:
        Node = max_node
    # 这里面使用networkx将图的信息存入
    G = nx.Graph()
    # Adj就是图的邻接表矩阵，是一个n*n大小的numpy矩阵，这里n是顶点的个数
    Adj = np.zeros([Node, Node], dtype=np.int32)
    # 遍历边的文件，将每条边存入networkx的图，以及邻接矩阵Adj所对应的位置(i, j)
    for i in range(edge.shape[0]):
        G.add_edge(edge[i][0], edge[i][1])
        if min_node == 0:
            Adj[edge[i][0], edge[i][1]] = 1
            Adj[edge[i][1], edge[i][0]] = 1
        else:
            Adj[edge[i][0] - 1, edge[i][1] - 1] = 1
            Adj[edge[i][1] - 1, edge[i][0] - 1] = 1
    # 转化成tensor
    return G, Adj, Node


In [17]:
G, Adj, Node = Read_graph('../../Graph/karate.edgelist')
Data = tf.data.Dataset.from_tensor_slices([Adj,Node])
for index in Test:
    adj_batch = Adj[index]
    adj_mat = adj_batch[:, index]
    b_mat = torch.ones_like(adj_batch)
    b_mat[adj_batch != 0] = 5
    print(b_mat)

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [15]:
Adj

<tf.Tensor: shape=(34, 34), dtype=int32, numpy=
array([[0, 1, 1, ..., 1, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 1, 1],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 0]], dtype=int32)>